# RL - Exercício 4 - Lunar Lander (Stable Baselines 3)

### Stable Baselines 3 - Treinamento, armazenamento e carga do modelo

Github Repo: [https://github.com/DLR-RM/stable-baselines3](https://github.com/DLR-RM/stable-baselines3)

Documentation is available online: [https://stable-baselines3.readthedocs.io/](https://stable-baselines3.readthedocs.io/)

## Trabalho desenvolvido durante o MBA em Data Science / IA na FIAP - 2022~2023.

## Prof. Felipe Teodoro

## Agente de Reinforcement Learning jogando Lunar Lander usando Deep Q-Learning

## Instalação das dependências e do Stable Baselines usando PIP

In [1]:
!apt-get install swig cmake
#!pip install setuptools==65.5.0

!pip install box2d-py
!pip install stable-baselines3[extra]

!pip install imageio
!pip install imageio_ffmpeg
!pip install pyglet==1.5.1

!apt update
!apt install ffmpeg xvfb
!pip install xvfbwrapper
!pip install pyvirtualdisplay


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.1).
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 18 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 2s (512 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 120875 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubunt

## Desabilitar _warnings_

In [2]:
# Não sumiram todos os warnings, mas mantivemos por legibilidade
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Importar política, Agente de Reinforcement Learning, etc

In [3]:
import gymnasium as gym
import numpy as np

from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback

from IPython import display
from IPython.display import HTML
#import pygame
from base64 import b64encode
import matplotlib.pyplot as plt
import imageio
from time import sleep
from tqdm import tqdm

import os

/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [4]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Criação do ambiente Gym e instanciar o agente

Para este exemplo, será usado o ambiente Lunar Lander do Gymnasium

"Aterrissar fora da plataforma de pouso é possível. O combustível é infinito, então um agente pode aprender a voar e então pousar em sua primeira tentativa. Quatro ações distintas estão disponíveis: não fazer nada, disparar o motor de orientação à esquerda, disparar o motor principal (para baixo), disparar o motor de orientação à direita."

Ambiente do módulo de pouso lunar: [https://gym.openai.com/envs/LunarLander-v2/](https://gym.openai.com/envs/LunarLander-v2/)

![Pouso Lunar](https://cdn-images-1.medium.com/max/960/1*f4VZPKOI0PYNWiwt0la0Rg.gif)

Escolhemos MlpPolicy porque a entrada do Lunar Lander é um vetor, e não imagens.

O tipo de ação a ser usada (discreta/contínua) será automaticamente deduzida do espaço de ação do ambiente.

In [5]:
env = gym.make("LunarLander-v2")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Callback para salvar o modelo quando a recompensa média melhora

Um monitor de recompensa acompanhará o treinamento, a cada 1024 passos, a média é calculada e se houver melhora em relação à última medição, o modelo atual será armazenado, substituindo o anterior.

In [6]:
class SaveOnBestRewardCallback(BaseCallback):
    def __init__(self, check_freq: int, model_dir: str, verbose=1):
        super(SaveOnBestRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.model_dir = model_dir
        # atribui -inf para a média inicial
        self.best_mean_reward = -float("inf")
        self.episode_rewards = []

    def _on_step(self) -> bool:
        if self.num_timesteps % self.check_freq == 0:
            # Calcula a recompensa média dos últimos check_freq timesteps
            mean_reward = np.mean(self.episode_rewards[-self.check_freq:])
            print('mean_reward :', mean_reward)
            print('self.best_mean_reward:', self.best_mean_reward)

            # Verifica se a recompensa média melhorou
            if mean_reward > self.best_mean_reward:
                self.best_mean_reward = mean_reward

                # Salva o modelo com a nova melhor média
                print('SAVING MODEL IN', self.model_dir)
                self.model.save(os.path.join(self.model_dir, "best_model"))

        return True

    def _on_rollout_end(self) -> None:
        # Registra a recompensa do episódio
        if self.model.ep_info_buffer:
            last_episode_info = self.model.ep_info_buffer[-1]
            last_episode_reward = last_episode_info.get('r', 0)
            self.episode_rewards.append(last_episode_reward)

# Diretório para salvar os modelos (padrão colab)
model_dir = "/content/"

# Criar o callback
checkpoint_callback = SaveOnBestRewardCallback(check_freq=1024, model_dir = model_dir)

model = DQN(
    'MlpPolicy',
    env,
    verbose = 1,
    exploration_fraction = 0.1,
    exploration_final_eps = 0.02,
    learning_starts = 1000,
    target_update_interval = 1000,
    gradient_steps = 1,
    train_freq = (4, 'step'),
    batch_size = 32,
    learning_rate = 0.001,
    gamma = 0.99,
    seed = 42,
    policy_kwargs = {'net_arch': [64, 64]},
    tensorboard_log = model_dir
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


## Avaliação do Modelo Inicial

In [7]:
# Ambiente separado para avaliação
eval_env = gym.make('LunarLander-v2')

# Agente aleatório, antes do treinamento
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward=-546.94 +/- 149.3965484381737


## Treinamento do Agente

In [8]:
# Treinamento do agente
model.learn(total_timesteps=int(10e5), callback=checkpoint_callback)

# Não será salvo o modelo após o treinamento, porque 

A saída de streaming foi truncada nas últimas 5000 linhas.
mean_reward : 120.90370797265626
self.best_mean_reward: 243.19696613671877
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 791      |
|    ep_rew_mean      | 65.5     |
|    exploration_rate | 0.02     |
| time/               |          |
|    episodes         | 1180     |
|    fps              | 843      |
|    time_elapsed     | 731      |
|    total_timesteps  | 617378   |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 3.2      |
|    n_updates        | 154094   |
----------------------------------
mean_reward : 159.96072388769534
self.best_mean_reward: 243.19696613671877
mean_reward : 102.61053874414064
self.best_mean_reward: 243.19696613671877
mean_reward : 51.82729515136719
self.best_mean_reward: 243.19696613671877
mean_reward : 86.92462164257813
self.best_mean_reward: 243.19696613671877
----------------------------------
| rollou

## Carga do Agente Treinado

In [9]:
del model
model = DQN.load("/content/best_model.zip")

## Avaliação do Agente Treinado

In [10]:
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=187.15 +/- 54.30916434026357


## Criar Vídeo do Agente Atuando

In [11]:
def convert_to_video(images, video_path):
  kargs = { 'macro_block_size': 1 }
  imageio.mimsave(video_path, [np.array(img) for i, img in enumerate(images)], fps=15, **kargs)

In [12]:
env = gym.make('LunarLander-v2', render_mode = "rgb_array")
obs = env.reset()
print(obs[0])

[-0.00752335  1.4029578  -0.7620645  -0.3539162   0.00872461  0.17261893
  0.          0.        ]


In [13]:
images = []
obs = obs[0]
for i in range(1000):

    action, _state  = model.predict(obs, deterministic=True)

    obs, reward, done, truncated, info = env.step(action)

    img = env.render()
    images.append(img)

    if done:
      obs = env.reset()
      obs = obs[0]

In [14]:
video_path = "replay.mp4"
convert_to_video(images, video_path)

In [15]:
# Show video
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)